** CaboChaインストール (windows)**

- 32bit版のpythonのcondaenvを作成。activateしたときにmecabとcabochaのbinがpathに追加されるように設定
- 32bit版のMecabをインストール（必ずCaboChaインストール前に上記のpathを通すことを含めてやる）
- CaboChaのexeからインストール（32bitのcondaenvをactivateしたうえで行う）
- 公式バインディングは評判も悪いし、インストールがそもそもできないのでbindingを使う（？）
VS community version を入れればよい？==>うまくいかず
http://isiz.hateblo.jp/entry/2015/08/22/Python%E3%81%A7Unable_to_find_vcvarsall.bat%E3%81%A8%E8%A8%80%E3%82%8F%E3%82%8C%E3%82%8B%E9%9A%9B%E3%81%AE%E3%81%BE%E3%81%A8%E3%82%81

Visual c++　2015 build tool?
https://blog.ionelmc.ro/2014/12/21/compiling-python-extensions-on-windows/




https://github.com/kenkov/cabocha

https://musaprg.hatenablog.com/entry/2018/02/09/005510


https://teratail.com/questions/80487


以下検証中:

https://qiita.com/mima_ita/items/161cd869648edb30627b

https://teratail.com/questions/80487


https://hayataka2049.hatenablog.jp/entry/2018/03/23/075140


In [1]:
import CaboCha


ModuleNotFoundError: No module named 'CaboCha'

In [1]:
from cabocha.analyzer import CaboChaAnalyzer

ModuleNotFoundError: No module named 'cabocha'

In [2]:
import cabocha


ModuleNotFoundError: No module named 'cabocha'